In [1]:
import torch
import torchvision
import numpy as np
from utils import *
import time
import os
import random
import torch.nn as nn
from models import *
from dataset import FlowSpeedData
import matplotlib.pyplot as plt
import cv2 as cv
import h5py

/home/ubuntu/anaconda3/envs/cs231n/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
files = ["2016-01-30--11-24-51.h5", "2016-01-30--13-46-00.h5", "2016-01-31--19-19-25.h5", 
"2016-02-02--10-16-58.h5", "2016-02-08--14-56-28.h5"]
flow_path = "data/flow_mul"
label_save_path = "data/label_mul"
model_path = "pre_trained.pt"
samples = 800

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model = torchvision.models.optical_flow.raft_small()
model.load_state_dict(torch.load(model_path))
model.to(device)
model.eval()

file_name = files[4]

video_path = os.path.join("camera", file_name)
with h5py.File(video_path, "r") as f:
    # Get the data
    frame_data = list(f['X'])
    print("Finished loading frames")
label_path = os.path.join("log", file_name)
with h5py.File(label_path, "r") as f:
    speed_abs_label = np.array(list(f['speed']))
    cam_ptr = np.array(list(f['cam1_ptr']))

np.random.seed(10086)
samples_id = np.random.randint(0, len(speed_abs_label) - 100, samples)

for i in range(samples):
    label_id = samples_id[i]
    frame_id = int(cam_ptr[label_id])
    image1 = frame_data[frame_id]
    image2 = frame_data[frame_id + 2]
    image1 = torch.from_numpy(image1).float().to(device)
    image2 = torch.from_numpy(image2).float().to(device)

    image1 = image1.unsqueeze(0)
    image2 = image2.unsqueeze(0)

    flow = model(image1, image2)
    flow = flow[-1].squeeze()

    flow = flow.cpu()
    write_flow(flow, os.path.join(flow_path, file_name + "_" + str(i).rjust(4, '0')+ ".flo"))

speeds = speed_abs_label[samples_id]
speeds_save = {
    "speed" : speeds
}
torch.save(speeds_save, os.path.join(label_save_path, file_name + "_speed.txt")) 
print(f'Finished {file_name}')


Finished loading frames
Finished 2016-02-08--14-56-28.h5
